<a href="https://colab.research.google.com/github/Eldorado7621/Hardware-Software-co-design-of-a-deep-learning-approach-for-detection-of-Arrhythmia-on-low-power-FPG/blob/main/cnn_cwt_an_fft_for_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tsaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Conv1D, Conv2D, MaxPool1D, GlobalMaxPool1D, MaxPool2D, GlobalMaxPool2D, Flatten, concatenate, Add, ReLU, Activation, BatchNormalization
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf

from scipy import signal

from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse

import pickle
import pywt
import matplotlib.pyplot as plt

import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

with open('drive/MyDrive/inputs_4class/inp_sig.pkl','rb') as f:
    X_all = pickle.load(f)
with open('drive/MyDrive/inputs_4class/labels.pkl','rb') as f:
    Y_all = pickle.load(f)

In [ ]:
Y_all = np.reshape(Y_all, ((Y_all.shape[0],1)))

all_data = np.concatenate((X_all,Y_all),axis=1)
np.random.shuffle(all_data)

indx_label4 = np.where(all_data[:,600] == 4)[0]
all_data = np.delete(all_data, indx_label4, 0)

###### 80% for train, 20% for test #########
sz = all_data.shape[0]

sz_08 = int(sz * 0.8)

In [ ]:
X_train_init = all_data[:sz_08,:600]
Y_train_init = all_data[:sz_08,600]

X_test_init = all_data[sz_08:,:600]
Y_test_init = all_data[sz_08:,600]

X_train_resamp = signal.resample(X_train_init, 128, axis=1)
X_test_resamp  = signal.resample(X_test_init, 128, axis=1)

del X_test_init
del X_train_init

In [ ]:
X_test_resamp[19]

array([-1.91687270e-02, -9.22563385e-02, -1.38220480e-01, -2.19474776e-01,
       -2.48736207e-01, -2.57146230e-01, -3.00333233e-01, -2.76902057e-01,
       -3.09712054e-01, -3.03604623e-01, -2.78108085e-01, -2.86834106e-01,
       -2.84476558e-01, -2.91025935e-01, -3.33566443e-01, -3.21417182e-01,
       -3.43626498e-01, -3.16897381e-01, -3.00003598e-01, -2.81076217e-01,
       -3.14078565e-01, -3.06125001e-01, -3.11395445e-01, -2.84961863e-01,
       -3.01319447e-01, -2.91390861e-01, -3.08075454e-01, -2.80456906e-01,
       -2.84874034e-01, -3.04270040e-01, -2.84161758e-01, -2.69421099e-01,
       -2.84299199e-01, -2.09337563e-01, -4.51794239e-02,  4.53100370e-01,
       -4.33004812e-03, -6.20892170e-01, -4.68037012e-01, -4.90600523e-01,
       -3.94612863e-01, -4.64958588e-01, -4.22890883e-01, -2.99594039e-01,
       -2.58982431e-01, -2.82897321e-01, -2.54304980e-01, -2.80375284e-01,
       -2.30404897e-01, -2.58858334e-01, -2.28766259e-01, -2.01841775e-01,
       -1.94223474e-01, -

In [ ]:
Y_test_init[19]

1.0

In [ ]:
for i in range(1000):
  if(Y_test_init[i]!=0.0):
    print("index %d value %d"%(i,Y_test_init[i] ))

index 12 value 2
index 19 value 1
index 30 value 2
index 31 value 1
index 35 value 3
index 36 value 1
index 47 value 2
index 83 value 2
index 115 value 3
index 128 value 2
index 144 value 1
index 154 value 3
index 173 value 1
index 183 value 1
index 201 value 3
index 206 value 1
index 239 value 1
index 247 value 2
index 248 value 1
index 250 value 1
index 252 value 3
index 282 value 2
index 319 value 1
index 366 value 2
index 384 value 2
index 396 value 2
index 414 value 2
index 430 value 1
index 465 value 2
index 478 value 2
index 553 value 2
index 554 value 2
index 597 value 1
index 605 value 2
index 669 value 2
index 673 value 2
index 678 value 2
index 706 value 2
index 714 value 2
index 754 value 1
index 771 value 1
index 773 value 2
index 775 value 1
index 790 value 1
index 795 value 2
index 796 value 2
index 807 value 2
index 811 value 1
index 824 value 2
index 839 value 1
index 844 value 1
index 846 value 2
index 855 value 2
index 886 value 1
index 889 value 2
index 896 value 1


In [ ]:
np.unique(Y_train_init)

array([0., 1., 2., 3.])

In [ ]:
np.histogram(Y_train_init)

(array([44571,     0,     0,  1299,     0,     0,  1930,     0,     0,
          228]),
 array([0. , 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3. ]))

In [ ]:
Y_test_init.shape

(12007,)

In [ ]:
wav_scale=64
scale = range(1,wav_scale+1)
fs = 128


**FFT AND IFFT**

In [ ]:


#create an array that stores the number of data point for each stage of the butterfly computation
def fft_no_of_stages(FFT_int_stage,fs):
    no_of_stages = 0
    value=fs
    while (value!=1):
        value=value>>1
        FFT_int_stage.append(value)
        no_of_stages=no_of_stages+1
	
    return no_of_stages

#compute the bit reversal
def bit_reversal(data_in,dit_inp,no_of_stages,fs,):
    #no_of_stages=fft_no_of_stages(FFT_int_stage)
    for j in range(0, fs):
        reversed=0
        for i in range(no_of_stages):
            reversed|=((j>>i)&1)<<(no_of_stages-i-1)
        
        dit_inp[j]=data_in[reversed]
        #dit_int.append(reversed)
      

#bit_reversal(dit_inp)

#compute the twiddle factor
def twiddle_factor(W,fs):
    for i in range(0,int(fs/2)):
        #print(2)
        #W.append(np.cos(2 * np.pi *i/ fs) ,-np.sin(2 * np.pi *i/ fs))
        W.append(complex(np.cos(2 * np.pi *i/ fs) ,-np.sin(2 * np.pi *i/ fs)))


def FFT0(FFT_stage,pass_check,index_shift,pass_shift,data_IN,data_OUT,W,fs):
    butterfly_span=0
    butterfly_pass=0
    for i in range(int(fs/2)):
        index = butterfly_span << index_shift
        Ulimit = butterfly_span + (butterfly_pass<<pass_shift)
        Llimit = Ulimit + FFT_stage
        Product = W[index] * data_IN[Llimit]
        data_OUT[Llimit] = data_IN[Ulimit]-Product
        data_OUT[Ulimit] = data_IN[Ulimit]+Product
        if (butterfly_span<(FFT_stage-1)):
            butterfly_span=butterfly_span+1
        elif (butterfly_pass<(pass_check-1)):
            butterfly_span = 0
            butterfly_pass=butterfly_pass+1
        else:
            butterfly_span = 0
            butterfly_pass=0
		
        
def fft(data_IN, fs):
    
    dit_inp=[]
    FFT_int_stage=[]
    W=[] #holds the twiddle factor
    data_OUT0=np.array([0.0]*fs,dtype=complex)
    data_OUT=np.array([0.0]*fs,dtype=complex)
    
    NO_STAGES=fft_no_of_stages(FFT_int_stage,fs)
    bit_reversal(data_IN, data_OUT,NO_STAGES,fs)
   
    twiddle_factor(W,fs)
    for i in range(0,NO_STAGES, 2):
        FFT0(FFT_int_stage[NO_STAGES-(i+1)],FFT_int_stage[i],NO_STAGES-(i+1),i+1,data_OUT,data_OUT0,W,fs);
        if(i+1<NO_STAGES):
            FFT0(FFT_int_stage[NO_STAGES-(i+2)],FFT_int_stage[i+1],NO_STAGES-(i+2),i+2,data_OUT0,data_OUT,W,fs);

    if (NO_STAGES%2==0):
        return data_OUT
    else:
        return data_OUT0

 
    

"""-------------------------IFFT SECTION BEGINS HERE----------------------------------"""

#compute the twiddle factor
def twiddle_factor_ifft(W,fft_len):
    for i in range(0,int(fft_len/2)):
        #print(2)
        #W.append(np.cos(2 * np.pi *i/ fft_len) ,-np.sin(2 * np.pi *i/ fft_len))
        W.append(complex(np.cos(2 * np.pi *i/ fft_len) ,np.sin(2 * np.pi *i/ fft_len)))

def ifft(data_INn,fft_len):    
    dit_inp=[]
    FFT_int_stage=[]
    W=[] #holds the twiddle factor   
    data_OUT0=np.array([0.0]*fft_len,dtype=complex)
    data_OUT=np.array([0.0]*fft_len,dtype=complex)
        
    no_of_stages=fft_no_of_stages(FFT_int_stage,fft_len)
    #bit_reversal(data_IN, data_OUT,FFT_int_stage)
    bit_reversal(data_INn, data_OUT,no_of_stages,fft_len)
   
    twiddle_factor_ifft(W,fft_len)
    for i in range(0,no_of_stages, 2):
        FFT0(FFT_int_stage[no_of_stages-(i+1)],FFT_int_stage[i],no_of_stages-(i+1),i+1,data_OUT,data_OUT0,W,fft_len);
        if(i+1<no_of_stages):
            FFT0(FFT_int_stage[no_of_stages-(i+2)],FFT_int_stage[i+1],no_of_stages-(i+2),i+2,data_OUT0,data_OUT,W,fft_len);
            
    if (no_of_stages%2==0):
        return data_OUT/fft_len
    else:
        return data_OUT0/fft_len



CWT FUNCTION

In [ ]:

def cwt(data, wavelet_name, sampling_frequency=1.):
    
    # Currently only supported for Morlet wavelets
    if wavelet_name == 'morl':
        #data -= np.mean(data)
        n_orig = data.size
        nv = 10
        ds = 1 / nv
        fs = sampling_frequency
        dt = 1 / fs

        # Pad data symmetrically
        padvalue = n_orig // 2
        x = np.concatenate((np.flipud(data[0:padvalue]), data, np.flipud(data[-padvalue:])))
        n = x.size

        # Define scales
        _, _, wavscales = getDefaultScales(wavelet_name, n_orig, ds)
        num_scales = wavscales.size

        # Frequency vector sampling the Fourier transform of the wavelet
        omega = np.arange(1, math.floor(n / 2) + 1, dtype=np.float64)
        omega *= (2 * np.pi) / n
        omega = np.concatenate((np.array([0]), omega, -omega[np.arange(math.floor((n - 1) / 2), 0, -1, dtype=int) - 1]))

        # Compute FFT of the (padded) time series
        #f = np.fft.fft(x)
        f= fft(x,n)


        # Loop through all the scales and compute wavelet Fourier transform
        psift, freq = waveft(wavelet_name, omega, wavscales)

        # Inverse transform to obtain the wavelet coefficients.        
        #b=np.ones([num_scales, 1])
        #kc=np.kron(b, f) 
        krp=np.kron(np.ones([num_scales, 1]), f) * psift
        # Inverse transform to obtain the wavelet coefficients.
        cwtcfs=np.array([[0.0]*krp.shape[1]]*krp.shape[0],dtype=complex)
        for i in range(krp.shape[0]):
           cwtcfs[i]= ifft(krp[i],n)
        #cwtcfs = np.fft.ifft(np.kron(np.ones([num_scales, 1]), f) * psift)
        cfs = cwtcfs[:, padvalue:padvalue + n_orig]
        freq = freq * fs

        return cfs, freq
    else:
        raise Exception


def getDefaultScales(wavelet, n, ds):
    
    wname = wavelet
    nv = 1 / ds

    if wname == 'morl':

        # Smallest useful scale (default 2 for Morlet)
        s0 = 2

        # Determine longest useful scale for wavelet
        #max_scale = n // (np.sqrt(2) * s0)
        max_scale=wav_scale
        if max_scale <= 1:
            max_scale = n // 2
        max_scale = np.floor(nv * np.log2(max_scale))
        a0 = 2 ** ds
        scales = s0 * a0 ** np.arange(0, wav_scale )
    else:
        raise Exception

    return s0, ds, scales


def waveft(wavelet, omega, scales):
    
    wname = wavelet
    num_freq = omega.size
    num_scales = scales.size
    wft = np.zeros([num_scales, num_freq])

    if wname == 'morl':
        gC = 6
        mul = 2
        for jj, scale in enumerate(scales):
            expnt = -(scale * omega - gC) ** 2 / 2 * (omega > 0)
            wft[jj, ] = mul * np.exp(expnt) * (omega > 0)

        fourier_factor = gC / (2 * np.pi)
        frequencies = fourier_factor / scales

    else:
        raise Exception

    return wft, frequencies

#fs = 1000
#t = np.linspace(0, 1, fs+1)
#x = np.cos(2*np.pi*32*t) * np.logical_and(t >= 0.1, t < 0.3) + np.sin(2*np.pi*64*t) * (t > 0.7)
#wgnNoise = 0.05 * np.random.standard_normal(t.shape)
#x += wgnNoise
#c, f = cwt(x, 'morl', sampling_frequency=fs)


In [ ]:

train_cwt_coeffs = np.zeros((len(X_train_resamp),len(scale),fs))
#train_cwt_coeffs, freqs = pywt.cwt(X_train, scale, 'morl')
for i in range(len(X_train_resamp)):
    #[train_cwt_coeffs[i], freqs] = pywt.cwt(X_train_resamp[i], scale, 'morl')
    print("here")
    print(i)
    [train_cwt_coeffs[i], _] = cwt(X_train_resamp[i], 'morl',fs)


test_cwt_coeffs = np.zeros((len(X_test_resamp),len(scale),fs))
#train_cwt_coeffs, freqs = pywt.cwt(X_train, scale, 'morl')
for i in range(len(X_test_resamp)):
    #[test_cwt_coeffs[i], freqs] = pywt.cwt(X_test_resamp[i], scale, 'morl')
    [test_cwt_coeffs[i], _] = cwt(X_test_resamp[i], 'morl',fs)

del X_train_resamp
del X_test_resamp

here
0
here
1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


here
2
here
3
here
4
here
5
here
6
here
7
here
8
here
9
here
10
here
11
here
12
here
13
here
14
here
15
here
16
here
17
here
18
here
19
here
20
here
21
here
22
here
23
here
24
here
25
here
26
here
27
here
28
here
29
here
30
here
31
here
32
here
33
here
34
here
35
here
36
here
37
here
38
here
39
here
40
here
41
here
42
here
43
here
44
here
45
here
46
here
47
here
48
here
49
here
50
here
51
here
52
here
53
here
54
here
55
here
56
here
57
here
58
here
59
here
60
here
61
here
62
here
63
here
64
here
65
here
66
here
67
here
68
here
69
here
70
here
71
here
72
here
73
here
74
here
75
here
76
here
77
here
78
here
79
here
80
here
81
here
82
here
83
here
84
here
85
here
86
here
87
here
88
here
89
here
90
here
91
here
92
here
93
here
94
here
95
here
96
here
97
here
98
here
99
here
100
here
101
here
102
here
103
here
104
here
105
here
106
here
107
here
108
here
109
here
110
here
111
here
112
here
113
here
114
here
115
here
116
here
117
here
118
here
119
here
120
here
121
here
122
here
123
here
124

KeyboardInterrupt: ignored

In [ ]:
np.savetxt("cwt_val_12.txt",train_cwt_coeffs[12],delimiter=",",fmt='%.5f')

In [ ]:
train_cwt_coeffs[1].shape

(64, 128)

In [ ]:
for i in range(1000):
  if(Y_test_init[i]!=0.0):
    print("index %d value %d"%(i,Y_test_init[i] ))

In [ ]:
def get_model():
    nclass = 4
    inp = tf.keras.layers.Input(shape=(32, 256, 1))
    img_1 = Conv2D(16,3, activation=tf.keras.activations.relu, padding="same")(inp)
    img_1 = MaxPool2D(pool_size=(2,2))(img_1)
    img_1 = Conv2D(32,3, activation=tf.keras.activations.relu, padding="same")(img_1)
    img_1 = MaxPool2D(pool_size=(2,2))(img_1)
    img_1 = Dropout(rate=0.2)(img_1)
    img_1 = Flatten()(img_1)
    dense_1 = Dense(16, activation=tf.keras.activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(8, activation=tf.keras.activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=tf.keras.activations.softmax, name="dense_3_mitbih")(dense_1)

    model = tf.keras.models.Model(inputs=inp, outputs=dense_1)
    opt = tf.keras.optimizers.Adam(0.001)
    #opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])
    #model.compile(optimizer=opt, loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [ ]:
model = get_model()
file_path = "ECG_mitbih.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = tf.keras.callbacks.EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(train_cwt_coeffs, Y_train_init, epochs=25, verbose=1, callbacks=callbacks_list, validation_split=0.1)
#model.load_weights(file_path)

pred_test = model.predict(test_cwt_coeffs)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test_init, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test_init, pred_test)

print("Test accuracy score : %s "% acc)

cm = confusion_matrix(Y_test_init, pred_test)
cm = cm / np.sum(cm,axis=0)

In [ ]:
MODEL_DIR = 'intmodel/'
MODEL_TFLITE = MODEL_DIR + 'vgint8model.tflite'

In [ ]:
model.save('intmodel/model.h5')
model.save('intmodel/model')

In [ ]:
train_cwt_coeffs[0].shape

In [ ]:
model.summary()

In [ ]:
train_cwt_coeffs[0].dtype

In [ ]:
import tensorflow as tf
# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_saved_model('intmodel/model')
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enforce integer only quantization
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.target_spec.supported_ops = tf.float32
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32
# Provide a representative dataset to ensure we quantize correctly.
def representative_dataset():
    for i in range(500):
        yield([np.float32(train_cwt_coeffs[i]).reshape(1, 32,256,1)])


converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)